In [0]:
%python
dbutils.widgets.text("catalog","new_catalog")
Catalog1=dbutils.widgets.get("catalog").strip()
dbutils.widgets.text("schema","default_schema")
Schema1=dbutils.widgets.get("schema").strip()

In [0]:
%python 
import json
json_obj=dbutils.notebook.run("/Workspace/my_databricks_learning/databricks_learning/Usecases/4_logistics_usecase_generic/generic_func_nb/common_config_nb",360,{"catalog_new":Catalog1,"schema_new":Schema1})

config_dict=json.loads(json_obj)
bronze_path=config_dict["bronze_path"]
source_path=config_dict["sourcedata_path"]
gold_path=config_dict["gold_path"]
gold_db=config_dict["gold_db"]
silver_db=config_dict["silver_db"]



In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {gold_db}.daily_dispatch_schedule AS 
SELECT
* FROM {gold_db}.logistics_shipment_gold_curated_tbl order by shipment_date asc, shipment_cost desc""")

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {gold_db}.critical_delays as
select * from  {gold_db}.logistics_shipment_gold_curated_tbl where shipment_status='DELAYED' order by shipment_date asc,shipment_cost desc limit 10""")

In [0]:
spark.sql(f"""CREATE TABLE if not exists {gold_db}.staff_with_shipments AS
SELECT
*
from {gold_db}.staff_gold_tbl t1 
inner join {gold_db}.logistics_shipment_gold_curated_tbl t2 
on t1.shipment_id=t2.log_shipment_id""")

In [0]:
spark.sql(f"""CREATE TABLE if not exists {gold_db}.invalid_driver_to_shipments
as 
select t2.* from {gold_db}.staff_gold_tbl t1 right join {gold_db}.logistics_shipment_gold_curated_tbl t2 on t1.shipment_id=t2.log_shipment_id where t1.shipment_id is null""")

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {gold_db}.geo_tagged_staff_data AS
select t1.*,t2.latitude,t2.longitude from {gold_db}.staff_gold_tbl t1 left join {silver_db}.master_city_df_tbl t2 on t1.origin_hub_city=t2.city_name""")

In [0]:


spark.sql(f"""create table if not exists {gold_db}.multi_level_report as
select sum(shipment_cost) as total_cost, origin_hub_city, vehicle_type from {gold_db}.staff_with_shipments group by cube(origin_hub_city,vehicle_type)""")